<a href="https://colab.research.google.com/github/psy794/stock_pjtt/blob/master/LR_%ED%8C%8C%EC%9D%B4%ED%94%84%EB%9D%BC%EC%9D%B8%2C_0710_(%EA%B2%B0%EA%B3%BC_0_1805743766).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [1]:
!pip install yfinance
!pip install finance-datareader
!pip install pykrx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 22.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import random
import os

import pickle

from tqdm import tqdm
from statsmodels.tsa.arima.model import ARIMA

#증권시장의 주식정보 스크래핑
#pip install pykrx
from pykrx import stock
from pykrx import bond

import yfinance as yf
import FinanceDataReader as fdr
import pykrx

import warnings
warnings.filterwarnings("ignore")

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

## Data Load

In [16]:
# train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')

In [17]:
# train.head(20)

,일자,종목코드,종목명,거래량,시가,고가,저가,종가
0,20210601,A060310,3S,166690,2890,2970,2885,2920
1,20210601,A095570,AJ네트웍스,63836,5860,5940,5750,5780
2,20210601,A006840,AK홀딩스,103691,35500,35600,34150,34400
3,20210601,A054620,APS,462544,14600,14950,13800,14950
4,20210601,A265520,AP시스템,131987,29150,29150,28800,29050
5,20210601,A211270,AP위성,623179,15300,16100,14950,16100
6,20210601,A027410,BGF,345706,7020,7150,7000,7050
7,20210601,A282330,BGF리테일,39293,182500,188500,181500,184000
8,20210601,A126600,BGF에코머티리얼즈,632510,6410,6540,6280,6400
9,20210601,A138930,BNK금융지주,1299625,7940,8000,7890,7990


In [19]:
# #종목코드에서 A떼고 Ticker컬럼에 넣기
# train['Ticker'] = train['종목코드'].str.replace('A', '')

In [ ]:

# # 각 종목별 OHLCV 데이터를 가져오기
# def get_ohlcv(tickers, start_date, end_date):
#     price_dataframes = []

#     for ticker in tickers:
#         print(f'Processing {ticker}')
#         price_df = stock.get_market_ohlcv_by_date(start_date, end_date, ticker)
#         price_df.reset_index(inplace=True)
#         price_df['종목코드'] = ticker

#         price_dataframes.append(price_df)
#     combined_price_df = pd.concat(price_dataframes, ignore_index=True)

#     return combined_price_df

# start_date = '20210601'
# end_date = '20230530'
# train_tickers = train['Ticker'].unique().tolist()  # 중복 제거하여 티커 값 리스트로 변환
# train = get_ohlcv(train_tickers, start_date, end_date)


In [ ]:
# train.head(10)

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,종목코드
0,2021-06-01,2890,2970,2885,2920,166690,487455970,1.04,060310
1,2021-06-02,2915,2975,2830,2900,134833,388795625,-0.68,060310
2,2021-06-03,2900,2925,2875,2900,144470,419668300,0.00,060310
3,2021-06-04,2930,3120,2920,2950,934224,2840373820,1.72,060310
4,2021-06-07,3000,3150,2955,3150,946560,2929678225,6.78,060310
5,2021-06-08,3155,3180,3035,3115,422752,1322783350,-1.11,060310
6,2021-06-09,3105,3280,3080,3105,655673,2097959840,-0.32,060310
7,2021-06-10,3105,3185,3010,3150,314027,975190275,1.45,060310
8,2021-06-11,3145,3175,3060,3085,302485,936045120,-2.06,060310
9,2021-06-14,3085,3100,3030,3080,174962,535956850,-0.16,060310


In [ ]:
# import pickle
# # 데이터프레임을 pickle 파일로 저장 (이유: 각 종목별 OHLCV 데이터를 가져오기가 오래걸려서 (45분정도 소요))
# pickle.dump(train, open('/content/drive/MyDrive/Colab Notebooks/train_data.pkl', 'wb'))

In [21]:
# 저장된 pickle 파일 불러오기
# loaded_train = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/train_data_df_new.pkl', 'rb'))
loaded_train = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/train_data.pkl', 'rb'))

In [22]:
train = loaded_train
train

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,종목코드
0,2021-06-01,2890,2970,2885,2920,166690,487455970,1.040000,060310
1,2021-06-02,2915,2975,2830,2900,134833,388795625,-0.680000,060310
2,2021-06-03,2900,2925,2875,2900,144470,419668300,0.000000,060310
3,2021-06-04,2930,3120,2920,2950,934224,2840373820,1.720000,060310
4,2021-06-07,3000,3150,2955,3150,946560,2929678225,6.780000,060310
...,...,...,...,...,...,...,...,...,...
987995,2023-05-23,6710,6710,6420,6430,40981,266207710,-4.170000,238490
987996,2023-05-24,6400,6490,6210,6320,19392,123635550,-1.710000,238490
987997,2023-05-25,6340,6400,6270,6330,6457,40862130,0.160000,238490
987998,2023-05-26,6330,6410,6300,6330,8905,56621540,0.000000,238490


# 파이프라인 (120기간으로 15일 예측 반복, OHLCV, MinMax, LR)

In [23]:
train.dtypes

날짜      datetime64[ns]
시가               int32
고가               int32
저가               int32
종가               int32
거래량              int32
거래대금             int64
등락률            float32
종목코드            object
dtype: object

In [ ]:
# ############################################
# ###### 학습시킬 기간과 예측 기간 확인 ######
# ############################################

# train_start_date = train['날짜'].min()
# train_end_date = train_start_date + pd.DateOffset(days=120)
# test_start_date = train_end_date + pd.DateOffset(days=1)
# test_end_date = test_start_date + pd.DateOffset(days=14)

# print(train_start_date)
# print(train_end_date)
# print(test_start_date)
# print(test_end_date)

# print("-----다음구간-----")
# # 다음 구간 설정
# train_start_date += pd.DateOffset(days=120)
# train_end_date += pd.DateOffset(days=120)
# test_start_date += pd.DateOffset(days=120)
# test_end_date += pd.DateOffset(days=120)

# print(train_start_date)
# print(train_end_date)
# print(test_start_date)
# print(test_end_date)

2021-06-01 00:00:00
2021-09-29 00:00:00
2021-09-30 00:00:00
2021-10-14 00:00:00
-----다음구간-----
2021-09-29 00:00:00
2022-01-27 00:00:00
2022-01-28 00:00:00
2022-02-11 00:00:00


In [31]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

# 특성 변환기 클래스 구현
class FeatureTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_transformed = X.copy()
        X_transformed[self.columns] = X_transformed[self.columns].astype(np.float32)
        return X_transformed

# Pipeline 생성
pipeline = Pipeline([
    ('preprocess', ColumnTransformer(transformers=[('num', FeatureTransformer(['시가', '고가', '저가', '거래량','거래대금'])),], remainder='passthrough')),
    ('scaling', MinMaxScaler()),
    ('model', LinearRegression())
])

# 반복적으로 학습 및 예측 수행
train_start_date = train['날짜'].min()
train_end_date = train_start_date + pd.DateOffset(days=120)
test_start_date = train_end_date + pd.DateOffset(days=1)
test_end_date = test_start_date + pd.DateOffset(days=14)

# 변수 초기화
total_return = 0.0
num_predictions = 0

# 결과를 저장할 DataFrame 생성
results_df = pd.DataFrame(columns=['종목코드', 'final_return'])

while test_end_date <= pd.to_datetime('2023-06-21'):  # 최종 예측일까지 반복
    # 학습 데이터 설정
    train_data = train[(train['날짜'] >= train_start_date) & (train['날짜'] <= train_end_date)]
    features = train_data.drop(['종가'], axis=1)
    labels = train_data['종가']

    # 날짜 데이터를 숫자형으로 변환
    features['날짜'] = features['날짜'].dt.strftime('%Y%m%d').astype(np.int32)

    # 스케일링
    scaler = MinMaxScaler()
    features_scaled = scaler.fit_transform(features)

    # 모델 학습
    model = LinearRegression()
    model.fit(features_scaled, labels)

    # 예측 데이터 설정
    test_data = train[(train['날짜'] >= test_start_date) & (train['날짜'] <= test_end_date)]
    test_features = test_data.drop(['종가'], axis=1)

    # 날짜 데이터를 숫자형으로 변환
    test_features['날짜'] = test_features['날짜'].dt.strftime('%Y%m%d').astype(np.int32)

    # 스케일링
    test_features_scaled = scaler.transform(test_features)

    # 예측
    predictions = model.predict(test_features_scaled)

    # 종목 코드별로 결과 저장
    for i, pred in enumerate(predictions):
        actual_close = test_data.iloc[i]["종가"]
        print(f'일자: {test_data.iloc[i]["날짜"]}, 종목 코드: {test_data.iloc[i]["종목코드"]}, 예측 종가: {pred}, 실제 종가: {actual_close}')
        if actual_close != 0:
            return_rate = (pred - actual_close) / actual_close
            total_return += return_rate
            num_predictions += 1

            # 결과를 DataFrame에 추가 (종목 코드별로 하나씩)
            if test_data.iloc[i]["종목코드"] not in results_df['종목코드'].values:
                results_df = results_df.append({'종목코드': test_data.iloc[i]["종목코드"], 'final_return': return_rate}, ignore_index=True)

    # 다음 구간 설정
    train_start_date += pd.DateOffset(days=120)
    train_end_date += pd.DateOffset(days=120)
    test_start_date += pd.DateOffset(days=120)
    test_end_date += pd.DateOffset(days=120)

# 최종 수익률 계산
final_return = total_return / num_predictions
print("최종 수익률:", final_return)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
일자: 2023-05-24 00:00:00, 종목 코드: 339950, 예측 종가: 2183.1615033507896, 실제 종가: 2030
일자: 2023-05-25 00:00:00, 종목 코드: 339950, 예측 종가: 2136.0002856868396, 실제 종가: 2005
일자: 2023-05-26 00:00:00, 종목 코드: 339950, 예측 종가: 2173.1940352670954, 실제 종가: 2010
일자: 2023-05-30 00:00:00, 종목 코드: 339950, 예측 종가: 2139.6054311425933, 실제 종가: 2000
일자: 2023-05-23 00:00:00, 종목 코드: 099190, 예측 종가: 16955.117672375916, 실제 종가: 16790
일자: 2023-05-24 00:00:00, 종목 코드: 099190, 예측 종가: 17238.291257255794, 실제 종가: 17240
일자: 2023-05-25 00:00:00, 종목 코드: 099190, 예측 종가: 17269.850129623883, 실제 종가: 17060
일자: 2023-05-26 00:00:00, 종목 코드: 099190, 예측 종가: 17233.822866030983, 실제 종가: 17400
일자: 2023-05-30 00:00:00, 종목 코드: 099190, 예측 종가: 17788.01310225965, 실제 종가: 17580
일자: 2023-05-23 00:00:00, 종목 코드: 289010, 예측 종가: 5092.271368043961, 실제 종가: 4960
일자: 2023-05-24 00:00:00, 종목 코드: 289010, 예측 종가: 5109.474504198883, 실제 종가: 4960
일자: 2023-05-25 00:00:00, 종목 코드: 289010, 예측 종가: 5052.288837824442, 실제 종가: 4915
일자: 2023-05-26 

,종목코드,final_return
0,060310,0.016892
1,095570,0.028771
2,006840,0.016163
3,054620,0.021089
4,265520,0.009463
...,...,...
1995,189980,0.050873
1996,000540,0.039120
1997,003280,-0.065183
1998,037440,0.053627


In [32]:
results_df

,종목코드,final_return
0,060310,0.016892
1,095570,0.028771
2,006840,0.016163
3,054620,0.021089
4,265520,0.009463
...,...,...
1995,189980,0.050873
1996,000540,0.039120
1997,003280,-0.065183
1998,037440,0.053627


In [33]:
results_df['순위'] = results_df['final_return'].rank(method='first', ascending=False).astype('int') # 각 순위를 중복없이 생성
results_df

,종목코드,final_return,순위
0,060310,0.016892,1062
1,095570,0.028771,731
2,006840,0.016163,1086
3,054620,0.021089,937
4,265520,0.009463,1333
...,...,...,...
1995,189980,0.050873,386
1996,000540,0.039120,530
1997,003280,-0.065183,1957
1998,037440,0.053627,364


In [34]:
results_df['종목코드'] = 'A' + results_df['종목코드']


In [35]:
sample_submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sample_submission.csv')
sample_submission

,종목코드,순위
0,A000020,1
1,A000040,2
2,A000050,3
3,A000070,4
4,A000080,5
...,...,...
1995,A375500,1996
1996,A378850,1997
1997,A383220,1998
1998,A383310,1999


In [36]:
baseline_submission = sample_submission[['종목코드']].merge(results_df[['종목코드', '순위']], on='종목코드', how='left')
baseline_submission

,종목코드,순위
0,A000020,318
1,A000040,72
2,A000050,1715
3,A000070,1870
4,A000080,1692
...,...,...
1995,A375500,1387
1996,A378850,430
1997,A383220,1896
1998,A383310,860


In [38]:
baseline_submission.to_csv('/content/drive/MyDrive/Colab Notebooks/baseline_submission0710.csv', index=False)

# 7월7일, 예측종가 단순 평균 낸 실수

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import pandas as pd

# 특성 변환기 클래스 구현
class FeatureTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_transformed = X.copy()
        X_transformed[self.columns] = X_transformed[self.columns].astype(np.float32)
        return X_transformed

# Pipeline 생성
pipeline = Pipeline([
    ('preprocess', ColumnTransformer(transformers=[('num', FeatureTransformer(['시가', '고가', '저가', '거래량'])),], remainder='passthrough')),
    ('scaling', MinMaxScaler()),
    ('model', LinearRegression())
])

# 결과를 저장할 DataFrame 생성
result_df = pd.DataFrame(columns=['종목코드', '예측종가'])

# 반복적으로 학습 및 예측 수행
train_start_date = train['날짜'].min()
train_end_date = train_start_date + pd.DateOffset(days=120)
test_start_date = train_end_date + pd.DateOffset(days=1)
test_end_date = test_start_date + pd.DateOffset(days=14)

while test_end_date <= pd.to_datetime('2023-06-21'):  # 최종 예측일까지 반복
    # 학습 데이터 설정
    train_data = train[(train['날짜'] >= train_start_date) & (train['날짜'] <= train_end_date)]
    features = train_data.drop(['종가'], axis=1)
    labels = train_data['종가']

    # 날짜 데이터를 숫자형으로 변환
    features['날짜'] = features['날짜'].dt.strftime('%Y%m%d').astype(np.int32)

    # 스케일링
    scaler = MinMaxScaler()
    features_scaled = scaler.fit_transform(features)

    # 모델 학습
    model = LinearRegression()
    model.fit(features_scaled, labels)

    # 예측 데이터 설정
    test_data = train[(train['날짜'] >= test_start_date) & (train['날짜'] <= test_end_date)]
    test_features = test_data.drop(['종가'], axis=1)

    # 날짜 데이터를 숫자형으로 변환
    test_features['날짜'] = test_features['날짜'].dt.strftime('%Y%m%d').astype(np.int32)

    # 스케일링
    test_features_scaled = scaler.transform(test_features)

    # 예측
    predictions = model.predict(test_features_scaled)

    # 결과 저장
    result_data = pd.DataFrame({'종목코드': test_data['종목코드'], '예측종가': predictions})
    result_df = pd.concat([result_df, result_data])

    # 다음 구간 설정
    train_start_date += pd.DateOffset(days=120)
    train_end_date += pd.DateOffset(days=120)
    test_start_date += pd.DateOffset(days=120)
    test_end_date += pd.DateOffset(days=120)

# 결과 출력
print(result_df)


          종목코드         예측종가
83      060310  4174.340680
84      060310  3956.911814
85      060310  3854.516886
86      060310  3537.499215
87      060310  4009.077539
...        ...          ...
987995  238490  6508.295261
987996  238490  6422.445347
987997  238490  6494.702054
987998  238490  6527.028834
987999  238490  8717.033492

[96000 rows x 2 columns]


In [ ]:
train.dtypes

In [ ]:
result_df2 = result_df.groupby('종목코드')['예측종가'].mean().reset_index()
result_df2.columns = ['종목코드', '예측종가평균']


In [ ]:
result_df2

,종목코드,예측종가평균
0,000020,12105.580090
1,000040,890.992140
2,000050,12854.168823
3,000070,81545.227000
4,000080,30234.614141
...,...,...
1995,375500,48202.538769
1996,378850,4618.352909
1997,383220,149716.141860
1998,383310,61318.135383


In [ ]:
result_df3['순위'] = result_df2['예측종가평균'].rank(method='first', ascending=False).astype('int') # 각 순위를 중복없이 생성
result_df3

,종목코드,예측종가평균,순위
0,000020,12105.580090,769
1,000040,890.992140,1956
2,000050,12854.168823,733
3,000070,81545.227000,110
4,000080,30234.614141,333
...,...,...,...
1995,375500,48202.538769,216
1996,378850,4618.352909,1399
1997,383220,149716.141860,51
1998,383310,61318.135383,168


In [ ]:
result_df3['종목코드'] = 'A' + result_df3['종목코드']
result_df3.drop('예측종가평균', axis=1, inplace=True)
result_df3


,종목코드,순위
0,A000020,769
1,A000040,1956
2,A000050,733
3,A000070,110
4,A000080,333
...,...,...
1995,A375500,216
1996,A378850,1399
1997,A383220,51
1998,A383310,168


In [ ]:
result_df3.to_csv('result_df3.csv', index=False)
##### 제출 점수 : -0.3650622054

#나중에 쓸 코드들

In [ ]:
# def get_mv(df):
#     """
#     이동평균을 계산하여 데이터프레임에 추가하는 함수

#     Parameters:
#         - df: 이동평균을 계산할 데이터프레임
#     Returns:
#         계산된 이동평균 값을 포함한 데이터프레임
#     """
#     grouped = df.groupby('종목코드')  # 종목별로 그룹화
#     df['Close_mv5'] = grouped['종가'].rolling(5, min_periods=5).mean().reset_index(0, drop=True) # 5일 이동평균 계산
#     df['Close_mv10'] = grouped['종가'].rolling(10, min_periods=10).mean().reset_index(0, drop=True)  # 10일 이동평균 계산
#     df['Close_mv20'] = grouped['종가'].rolling(20, min_periods=20).mean().reset_index(0, drop=True)  # 20일 이동평균 계산

#     return df


# train = get_mv(train)
# train

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,종목코드,Close_mv5,Close_mv10,Close_mv20
0,2021-06-01,2890,2970,2885,2920,166690,487455970,1.040000,060310,NaN,NaN,NaN
1,2021-06-02,2915,2975,2830,2900,134833,388795625,-0.680000,060310,NaN,NaN,NaN
2,2021-06-03,2900,2925,2875,2900,144470,419668300,0.000000,060310,NaN,NaN,NaN
3,2021-06-04,2930,3120,2920,2950,934224,2840373820,1.720000,060310,NaN,NaN,NaN
4,2021-06-07,3000,3150,2955,3150,946560,2929678225,6.780000,060310,2964.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
987995,2023-05-23,6710,6710,6420,6430,40981,266207710,-4.170000,238490,6540.0,6416.0,6440.5
987996,2023-05-24,6400,6490,6210,6320,19392,123635550,-1.710000,238490,6548.0,6399.0,6429.5
987997,2023-05-25,6340,6400,6270,6330,6457,40862130,0.160000,238490,6502.0,6383.0,6421.0
987998,2023-05-26,6330,6410,6300,6330,8905,56621540,0.000000,238490,6424.0,6393.0,6419.5


In [ ]:
# def get_obv(train):#OBV 컬럼에 추가하기
#     #종목코드 2000가지를 가져오자
#     stock_code = train['종목코드'].unique()

#     #데이터 프레임에 틀 만들기
#     train['OBV'] = 0
#     train['OBV_EMA'] = 0

#     for code in tqdm(stock_code): #각 종목별, 총 2000회 반복
#         df_train = train[train['종목코드'] == code]
#         OBV = []
#         OBV.append(0) #[-1]을 사용하기 위해 초기 값을 넣어준다.
#         for i in range(1, len(df_train['종가'])): #기간 만큼 돌려준다.
#             if df_train['종가'].iloc[1] > df_train['종가'].iloc[i-1]: #현재 종가가 전일 종가보다 크다면
#                 OBV.append(OBV[-1] + df_train['거래량'].iloc[i]) #OBV 제일 마지막 값에 현재일 거래량 값을 더한 값을 OBV에 추가.
#             elif df_train['종가'].iloc[1] < df_train['종가'].iloc[i-1]: #현재 종가가 전일 종가보다 작다면
#                 OBV.append(OBV[-1] - df_train['거래량'].iloc[i]) #OBV 제일 마지막 값에 현재일 거래량 값을 빼준 값을 OBV에 추가.
#             else: #종가가 동일하다면
#                 OBV.append(OBV[-1]) #OBV 제일 마지막 값을 다시 한번 OBV에 추가.

#         train.loc[train['종목코드'] == code, 'OBV'] = OBV #OBV 리스트 안의 값을 코드에 넣어주자.
#         train.loc[train['종목코드'] == code,'OBV_EMA'] = train[train['종목코드'] == code]['OBV'].ewm(com=20).mean() #OBV의 이동평균 값을 넣어주자.
#         return train


# # forex 데이터
# def get_forex_index(df):
#     start_date_yf = '2021-06-01'
#     end_date_yf = '2023-05-31'

#     start_date = '20210601'
#     end_date = '20230531'

#     forex_index_data = yf.download(["USDKRW=X", "USDAUD=X", "USDJPY=X", "EURUSD=X", "CNY=X", "^GSPC", "^DJI", "^IXIC", "^STOXX50E",
#                                    "^SOX",  "000001.SS", "000300.SS", "MME=F", "^TNX"],
#                                   start=start_date_yf, end=end_date_yf, rounding=True)

#     tmp_forex_index = forex_index_data["Close"]
#     tmp_forex_index.index = pd.to_datetime(tmp_forex_index.index)
#     tmp_forex_index = tmp_forex_index[(tmp_forex_index.index >= pd.to_datetime(start_date)) &
#                                       (tmp_forex_index.index <= pd.to_datetime(end_date))]
#     tmp_forex_index.columns = ["sse_composite_index", "csi300_index", "usdtocny", "eurtousd", "msci_emerging",
#                                "usdtoaud", "usdtojpy", "usdtokrw",
#                                "dow", "snp500", "nasdaq", "semicon_index", "euro50", "us10y_tsy"]

#     tmp_forex_index.reset_index(drop=False, inplace=True)

#     merged_df = df.merge(tmp_forex_index, left_on='날짜', right_on='Date', how='left')
#     merged_df.drop(columns=['Date'], inplace=True)
#     return merged_df



# #스토캐스틱 fast 지표
# def sto_fast_logic(df_fast):
#     n = 14
#     m = 5

#     #n일중 최고가
#     ndays_high = df_fast['고가'].rolling(window = n, min_periods = 1).max()
#     #n일중 최저가
#     ndays_low = df_fast['저가'].rolling(window = n, min_periods = 1).min()
#     #Fast%K 계산
#     fast_k = ((df_fast['종가'] - ndays_low) / (ndays_high - ndays_low)) * 100
#     #Fast%D 계산
#     fast_d = fast_k.ewm(span = m).mean()

#     return (fast_k, fast_d)

# def get_sto_fast(df):
#     df['fast_k'] = 0
#     df['fast_d'] = 0
#     stock_code = df['종목코드'].unique()

#     for code in tqdm(stock_code): #각 종목별, 총 2000회 반복
#         df_fast = df[df['종목코드'] == code]
#         (fast_k, fast_d) = sto_fast_logic(df_fast)
#         df.loc[df['종목코드'] == code, 'fast_k'] = fast_k
#         df.loc[df['종목코드'] == code,'fast_d'] = fast_d
#     return df

# #스토캐스틱 slow 지표
# def sto_slow_logic(df_slow):
#     n = 14
#     m = 5
#     t = 5

#     #n일중 최고가
#     ndays_high = df_slow['고가'].rolling(window = n, min_periods = 1).max()
#     #n일중 최저가
#     ndays_low = df_slow['저가'].rolling(window = n, min_periods = 1).min()
#     #Fast%K 계산
#     fast_k = ((df_slow['종가'] - ndays_low) / (ndays_high - ndays_low)) * 100
#     #Slow%K 계산
#     slow_k = fast_k.ewm(span = m).mean()
#     #Slow%D 계산
#     slow_d = slow_k.ewm(span = t).mean()

#     return (slow_k, slow_d)

# def get_sto_slow(df):
#     df['slow_k'] = 0
#     df['slow_d'] = 0
#     stock_code = df['종목코드'].unique()

#     for code in tqdm(stock_code): #각 종목별, 총 2000회 반복
#         df_slow = df[df['종목코드'] == code]
#         (slow_k, slow_d) = sto_fast_logic(df_slow)
#         df.loc[df['종목코드'] == code, 'slow_k'] = slow_k
#         df.loc[df['종목코드'] == code,'slow_d'] = slow_d
#     return df


# train = get_obv(train)
# train = get_forex_index(train)
# train = get_sto_fast(train)


  0%|          | 0/2000 [00:00<?, ?it/s]


[*********************100%***********************]  14 of 14 completed


100%|██████████| 2000/2000 [09:06<00:00,  3.66it/s]


In [ ]:
# train.isnull().sum()

날짜                         0
시가                         0
고가                         0
저가                         0
종가                         0
거래량                        0
거래대금                       0
등락률                        0
종목코드                       0
Close_mv5               8000
Close_mv10             18000
Close_mv20             38000
OBV                        0
OBV_EMA                    0
sse_composite_index    50000
csi300_index           50000
usdtocny                   0
eurtousd                   0
msci_emerging          22000
usdtoaud                   0
usdtojpy                   0
usdtokrw                   0
dow                    34000
snp500                 34000
nasdaq                 34000
semicon_index          34000
euro50                 20000
us10y_tsy              34000
fast_k                     1
fast_d                 14402
dtype: int64

In [ ]:
# pickle.dump(train, open('/content/drive/MyDrive/Colab Notebooks/train_data_df2.pkl', 'wb'))

In [ ]:
loaded_train_df = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/train_data_df2.pkl', 'rb'))

In [ ]:
loaded_train_df

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,종목코드,Close_mv5,...,usdtojpy,usdtokrw,dow,snp500,nasdaq,semicon_index,euro50,us10y_tsy,fast_k,fast_d
0,2021-06-01,2890,2970,2885,2920,166690,487455970,1.040000,060310,NaN,...,109.493,1106.9800,34575.31,4202.04,13736.48,3174.32,4071.75,1.615,41.176471,41.176471
1,2021-06-02,2915,2975,2830,2900,134833,388795625,-0.680000,060310,NaN,...,109.493,1107.2100,34600.38,4208.12,13756.33,3196.98,4088.50,1.591,48.275862,45.436105
2,2021-06-03,2900,2925,2875,2900,144470,419668300,0.000000,060310,NaN,...,109.595,1109.8800,34577.04,4192.85,13614.51,3139.19,4079.24,1.625,48.275862,46.781253
3,2021-06-04,2930,3120,2920,2950,934224,2840373820,1.720000,060310,NaN,...,110.266,1115.4399,34756.39,4229.89,13814.49,3214.14,4089.38,1.560,41.379310,44.537369
4,2021-06-07,3000,3150,2955,3150,946560,2929678225,6.780000,060310,2964.0,...,109.617,1109.6801,34630.24,4226.52,13881.72,3191.66,4097.65,1.569,100.000000,65.828711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987995,2023-05-23,6710,6710,6420,6430,40981,266207710,-4.170000,238490,6540.0,...,138.672,1313.8300,33055.51,4145.58,12560.25,3177.44,4342.38,3.698,55.789474,64.061314
987996,2023-05-24,6400,6490,6210,6320,19392,123635550,-1.710000,238490,6548.0,...,138.598,1319.8400,32799.92,4115.24,12484.16,3124.09,4263.74,3.719,44.210526,57.444385
987997,2023-05-25,6340,6400,6270,6330,6457,40862130,0.160000,238490,6502.0,...,139.283,1318.5200,32764.65,4151.28,12698.09,3336.72,4269.64,3.814,45.263158,53.383976
987998,2023-05-26,6330,6410,6300,6330,8905,56621540,0.000000,238490,6424.0,...,140.017,1330.8900,33093.34,4205.45,12975.69,3545.67,4337.50,3.810,45.263158,50.677037


In [ ]:
train = loaded_train_df

In [ ]:
print(train.columns)


Index(['날짜', '시가', '고가', '저가', '종가', '거래량', '거래대금', '등락률', '종목코드', 'Close_mv5',
       'Close_mv10', 'Close_mv20', 'OBV', 'OBV_EMA', 'sse_composite_index',
       'csi300_index', 'usdtocny', 'eurtousd', 'msci_emerging', 'usdtoaud',
       'usdtojpy', 'usdtokrw', 'dow', 'snp500', 'nasdaq', 'semicon_index',
       'euro50', 'us10y_tsy', 'fast_k', 'fast_d'],
      dtype='object')


In [ ]:
train

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,종목코드,Close_mv5,...,usdtojpy,usdtokrw,dow,snp500,nasdaq,semicon_index,euro50,us10y_tsy,fast_k,fast_d
0,2021-06-01,2890,2970,2885,2920,166690,487455970,1.040000,060310,NaN,...,109.493,1106.9800,34575.31,4202.04,13736.48,3174.32,4071.75,1.615,41.176471,41.176471
1,2021-06-02,2915,2975,2830,2900,134833,388795625,-0.680000,060310,NaN,...,109.493,1107.2100,34600.38,4208.12,13756.33,3196.98,4088.50,1.591,48.275862,45.436105
2,2021-06-03,2900,2925,2875,2900,144470,419668300,0.000000,060310,NaN,...,109.595,1109.8800,34577.04,4192.85,13614.51,3139.19,4079.24,1.625,48.275862,46.781253
3,2021-06-04,2930,3120,2920,2950,934224,2840373820,1.720000,060310,NaN,...,110.266,1115.4399,34756.39,4229.89,13814.49,3214.14,4089.38,1.560,41.379310,44.537369
4,2021-06-07,3000,3150,2955,3150,946560,2929678225,6.780000,060310,2964.0,...,109.617,1109.6801,34630.24,4226.52,13881.72,3191.66,4097.65,1.569,100.000000,65.828711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987995,2023-05-23,6710,6710,6420,6430,40981,266207710,-4.170000,238490,6540.0,...,138.672,1313.8300,33055.51,4145.58,12560.25,3177.44,4342.38,3.698,55.789474,64.061314
987996,2023-05-24,6400,6490,6210,6320,19392,123635550,-1.710000,238490,6548.0,...,138.598,1319.8400,32799.92,4115.24,12484.16,3124.09,4263.74,3.719,44.210526,57.444385
987997,2023-05-25,6340,6400,6270,6330,6457,40862130,0.160000,238490,6502.0,...,139.283,1318.5200,32764.65,4151.28,12698.09,3336.72,4269.64,3.814,45.263158,53.383976
987998,2023-05-26,6330,6410,6300,6330,8905,56621540,0.000000,238490,6424.0,...,140.017,1330.8900,33093.34,4205.45,12975.69,3545.67,4337.50,3.810,45.263158,50.677037


In [ ]:
train['시가총액'] = None  # 시가총액 컬럼 추가

In [ ]:
def add_market_cap(df):
    df['시가총액'] = 0

    for index, row in df.iterrows():
        ticker = row['종목코드']
        date = row['날짜'].strftime('%Y-%m-%d')  # 날짜 형식 변환
        market_cap = stock.get_market_cap(date, date, ticker)
        df.at[index, '시가총액'] = market_cap['시가총액']
    return df

train = add_market_cap(train)
train

In [ ]:
%pip install seaborn
import seaborn as sns

import matplotlib.pyplot as plt

plt.rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False
# 시각화되는 내용들을 고해상도로 확인하기 위한 설정
plt.rcParams['figure.dpi'] = 200

In [ ]:
train_df = train.copy()
train_df.drop(columns = ['종가'], inplac = True)

train_df_cols = train_df.columns[:]
train_df_cor = abs(train_df[train_df_cols].corr(method='searman'))
tain_df_cor = pd.DataFrame(train_df_cor['시가']).sort_values('시가', ascending=False)
train_df_cor.columns=['corr']

sns.heatmap(abs(train_df[train_df_cols].corr(method='spearman')), annot=True, annot_kws={'size':4})

In [ ]:
import random

# 10개의 샘플 종목 코드 선택
sample_codes = random.sample(list(train['종목코드'].unique()), 10)

for code in sample_codes:
    # 종목 코드에 해당하는 데이터 추출
    subset = train[train['종목코드'] == code].copy()

    # '종가' 열 제거
    subset.drop(columns=['종가'], inplace=True)

    # 상관관계 계산
    correlations = subset.corr(method='spearman')

    # 원하는 작업 수행 (예: 특정 열과의 상관계수 확인, 시각화 등)
    print(f"종목 코드 {code}에 대한 상관계수:")
    print(correlations)
    print("========================================")


종목 코드 032620에 대한 상관계수:
                           시가        고가        저가       거래량      거래대금  \
시가                   1.000000  0.991597  0.996351  0.480898  0.614915   
고가                   0.991597  1.000000  0.990506  0.529610  0.661240   
저가                   0.996351  0.990506  1.000000  0.458634  0.595534   
거래량                  0.480898  0.529610  0.458634  1.000000  0.983271   
거래대금                 0.614915  0.661240  0.595534  0.983271  1.000000   
등락률                 -0.051210  0.012747 -0.006993  0.070894  0.077438   
Close_mv5            0.992041  0.985691  0.992003  0.454181  0.589775   
Close_mv10           0.979302  0.971600  0.979087  0.419209  0.555838   
Close_mv20           0.951519  0.942505  0.951837  0.369389  0.504144   
OBV                       NaN       NaN       NaN       NaN       NaN   
OBV_EMA                   NaN       NaN       NaN       NaN       NaN   
sse_composite_index  0.750581  0.756701  0.747625  0.360149  0.463788   
csi300_index         0.83992

In [ ]:
!apt-get update -qq
!apt-get install fonts-nanum* -qq

Selecting previously unselected package fonts-nanum-extra.
(Reading database ... 123071 files and directories currently installed.)
Preparing to unpack .../fonts-nanum-extra_20180306-3_all.deb ...
Unpacking fonts-nanum-extra (20180306-3) ...
Selecting previously unselected package fonts-nanum.
Preparing to unpack .../fonts-nanum_20180306-3_all.deb ...
Unpacking fonts-nanum (20180306-3) ...
Selecting previously unselected package fonts-nanum-coding.
Preparing to unpack .../fonts-nanum-coding_2.5-2_all.deb ...
Unpacking fonts-nanum-coding (2.5-2) ...
Selecting previously unselected package fonts-nanum-eco.
Preparing to unpack .../fonts-nanum-eco_1.000-7_all.deb ...
Unpacking fonts-nanum-eco (1.000-7) ...
Setting up fonts-nanum-extra (20180306-3) ...
Setting up fonts-nanum (20180306-3) ...
Setting up fonts-nanum-coding (2.5-2) ...
Setting up fonts-nanum-eco (1.000-7) ...
Processing triggers for fontconfig (2.13.1-2ubuntu3) ...


In [ ]:
# 일단 import
%matplotlib inline

import matplotlib as mpl  # 기본 설정 만지는 용도
import matplotlib.pyplot as plt  # 그래프 그리는 용도
import matplotlib.font_manager as fm  # 폰트 관련 용도
print(mpl.__version__)
print(mpl.__file__)
print(mpl.get_configdir())
print(mpl.get_cachedir())

# 한번 프린트 해보자
# 체크해보면 폰트 개수가 늘어났다
sys_font=fm.findSystemFonts()
print(f"sys_font number: {len(sys_font)}")

nanum_font = [f for f in sys_font if 'Nanum' in f]
print(f"nanum_font number: {len(nanum_font)}")

nanum_font

3.7.1
/usr/local/lib/python3.10/dist-packages/matplotlib/__init__.py
/root/.config/matplotlib
/root/.cache/matplotlib
sys_font number: 48
nanum_font number: 31


['/usr/share/fonts/truetype/nanum/NanumBarunGothicLight.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothicEcoBold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothicLight.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothic.ttf',
 '/usr/share/fonts/truetype/nanum/NanumBarunpenB.ttf',
 '/usr/share/fonts/truetype/nanum/NanumMyeongjoEco.ttf',
 '/usr/share/fonts/truetype/nanum/NanumMyeongjo.ttf',
 '/usr/share/fonts/truetype/nanum/NanumBrush.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothicEcoExtraBold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumBarunGothicBold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumPen.ttf',
 '/usr/share/fonts/truetype/nanum/NanumSquareRoundEB.ttf',
 '/usr/share/fonts/truetype/nanum/NanumMyeongjoExtraBold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumBarunGothicUltraLight.ttf',
 '/usr/share/fonts/truetype/nanum/NanumSquareB.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothicCoding-Bold.ttf',
 '/usr/share/fon

In [ ]:
# 현재 설정되어 있는 폰트 사이즈와 글꼴을 알아보자
!python --version
def current_font():
  print(f"설정 폰트 글꼴: {plt.rcParams['font.family']}, 설정 폰트 사이즈: {plt.rcParams['font.size']}")  # 파이썬 3.6 이상 사용가능하다

current_font()

Python 3.10.12
설정 폰트 글꼴: ['NanumGothic'], 설정 폰트 사이즈: 10.0


In [ ]:
path = '/usr/share/fonts/truetype/nanum/NanumGothicEco.ttf'  # 설치된 나눔글꼴중 원하는 녀석의 전체 경로를 가져오자
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)

NanumGothic Eco


In [ ]:
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_df = pd.DataFrame(columns=['종목코드', 'VIF Factor'])

# 종목 코드별로 VIF 계산
code_list = train['종목코드'].unique()

for code in code_list:
    # 종목 코드에 해당하는 데이터 추출
    subset = train[train['종목코드'] == code].copy()

    # '종가' 열 제거
    subset.drop(columns=['종가'], inplace=True)

    # 숫자형 열 추출
    numeric_cols = subset.select_dtypes(include=[np.number]).columns

    if len(numeric_cols) > 0:
        # 결측값 처리
        subset[numeric_cols] = subset[numeric_cols].replace([np.inf, -np.inf], np.nan)
        subset.dropna(subset=numeric_cols, inplace=True)

        # VIF 계산을 위해 숫자형으로 변환
        subset[numeric_cols] = subset[numeric_cols].astype(float)

        # VIF 계산
        vif_factors = []
        for i in range(len(numeric_cols)):
            try:
                vif = variance_inflation_factor(subset[numeric_cols].values, i)
                vif_factors.append(vif)
            except ValueError:
                # 계산 중에 오류가 발생하면 VIF 값을 0으로 설정
                vif_factors.append(0)

        # 결과 저장
        vif_df = vif_df.append({'종목코드': code, 'VIF Factor': vif_factors}, ignore_index=True)

# VIF 값 기준으로 정렬
vif_df.sort_values(by='VIF Factor', ascending=False, inplace=True)

print(vif_df)


        종목코드                                         VIF Factor
107   003620  [4503599627370496.0, 818836295885544.8, 237031...
751   029780  [72048.48471283943, 89408.23355442003, 77198.4...
299   024110  [68215.74972255394, 62933.98414522768, 56298.5...
762   002810  [66491.16078298284, 68594.29479188281, 53570.2...
1821  002960  [65440.58820289565, 44776.87743496814, 75207.5...
...      ...                                                ...
1490  033340  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1420  178780  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1059  263540  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1377  160600  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
619   257370  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...

[2000 rows x 2 columns]


In [ ]:
vif_df.tail(30)

,종목코드,VIF Factor
555,084650,"[177.3930584309655, 115.23591543360838, 131.10..."
860,258830,"[173.90020037332292, 132.07644619665055, 84.20..."
1298,047400,"[173.69171345121197, 133.23713360801452, 150.8..."
500,005290,"[171.03236003496767, 123.82848899572622, 127.8..."
105,151860,"[170.9086720847617, 149.91065811280254, 105.88..."
72,028300,"[169.13715263692347, 208.86495055298144, 166.8..."
110,001390,"[158.70983114736535, 115.87520666439626, 95.42..."
201,048550,"[156.9698554305359, 104.20429760768288, 128.18..."
1737,047050,"[154.92303212754916, 146.7296665237702, 139.61..."
1697,081150,"[148.65325816062966, 114.08995530993818, 130.3..."
